In [ ]:
%pip install -r requirements.txt

In [ ]:
# scrape https://www.poetryfoundation.org/poems/browse#page=1&sort_by=recently_added for 40,000 poems?

In [3]:
import pandas as pd
import tensorflow as tf
import numpy as np
import string
import logging

In [27]:
df = pd.read_pickle('poems.pickle')
print(len(df))

17369


In [28]:
for i in range(len(df)):
    if df.content[i] == '':
        df.drop(index=i, inplace=True)
print(len(df))

16502


In [29]:
df.reset_index(drop=True, inplace=True)

In [31]:
df.to_pickle('clean_poems.pickle')

In [39]:
df.tags[0] = []

In [40]:
df

,title,author,content,tags
0,“I write until my face is erased”,Beatriz Miralles de Imperial,I write until my face is erased\nonly who I am...,[]
1,Entangled,Sherwin Bitsui,"He sees birds overhead,\neggshells seared\nto ...",[]
2,From “Postcards”,Bert Meyers,"ocean\n\nThere it is: an immense, gray, agitat...",[]
3,First Kiss/Under Capitalism,Omotara James,When I learned I had power to build\non this m...,[]
4,"Harold Norse says, “Poetry meant being a sissy...",Tyler Raso,"As a poet and a faggot, I\nhave often been app...",[]
...,...,...,...,...
16497,Sylvester’s Dying Bed,Langston Hughes,I woke up this mornin’\n’Bout half-past three....,"[Living, Death, Sorrow & Grieving, Love, Heart..."
16498,At Melville's Tomb,Hart Crane,"Often beneath the wave, wide from this ledge\n...",[]
16499,A Woman and Mountains,Helen Hoyt,"I am rounded, billowed out, hunched like you;\...","[Nature, Landscapes & Pastorals, Social Commen..."
16500,Purple,Margaret Steele Anderson,A pigeon walking dainty in the street;\nThe mo...,[]


In [43]:
df = pd.read_pickle('clean_poems.pickle')

In [44]:
for i in range(len(df)):
    if df.tags[i] == []:
        df.tags[i] = ''
    else:
        df.tags[i] = ', '.join(df.tags[i])

In [46]:
df.to_csv('clean_poems.csv')

In [26]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\olive\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [27]:
Poems = df.content.map(lambda x: nltk.tokenize.word_tokenize(x))

In [28]:
Poems[1]

['He',
 'sees',
 'birds',
 'overhead',
 ',',
 'eggshells',
 'seared',
 'to',
 'their',
 'wire-wrapped',
 'claws',
 '.',
 'Their',
 'flock',
 '’',
 's',
 'light',
 'dims',
 'when',
 'cedar',
 'smoke',
 'sniffs',
 'hunger',
 'on',
 'text',
 'messages',
 'flailing',
 'in',
 'the',
 'sage',
 'brush',
 '.',
 'He',
 'unbraids',
 'rain',
 'from',
 'his',
 'hair',
 ',',
 'notices',
 ':',
 'each',
 'strand',
 'slows',
 'to',
 'a',
 'screech',
 'when',
 'glimpsed',
 'by',
 'a',
 'steer',
 'ducking',
 'the',
 'lasso',
 '’',
 's',
 'widening',
 'maw',
 '.',
 'With',
 'flames',
 'belted',
 'to',
 'his',
 'waist',
 ',',
 'he',
 'pins',
 'moth',
 'wings',
 'to',
 'each',
 'word',
 'cresting',
 '.',
 'Eyes',
 'shell-white',
 ',',
 'he',
 'ignites',
 'his',
 'gas-soaked',
 'shoes—',
 'emergency',
 'lights',
 'flare',
 'in',
 'every',
 'limb',
 'of',
 'his',
 'waking',
 '.']

In [29]:
def BiGram(dic, poem):
    a = poem
    b = poem[1:]
    for i in range(len(b)):
        if(a[i] in dic):
            if(b[i] in dic[a[i]]):
                dic[a[i]][b[i]] = dic[a[i]][b[i]]+1
            else:
                dic[a[i]][b[i]] = 1
        else:
            dic[a[i]]={b[i]:1}
            
    return(dic)

In [30]:
dicBi = {} # The dictionary for the BiGram model
for poem in Poems: #Feed it all the poems
    dicBi = BiGram(dicBi, poem) #Build the model!

In [31]:
def Next_word(word, dic):
    Max = 0
    next_word = ""
    for i,j in list(dic[word].items()):
        if(j>Max):
            Max = j
            next_word = i
    return(next_word)

In [35]:
prev_word = "huh"
generate_str = prev_word
for i in range(20):
    next_word = Next_word(prev_word, dicBi)
    prev_word = next_word
    generate_str = generate_str+" "+ next_word

In [36]:
generate_str

'huh ? I ’ s a man , and the way to the way to the way to the way to'

In [259]:
def NGram(dictionary, poem, nGram):
    Number_of_Ngrams = len(poem)-nGram+1

    for position in range(Number_of_Ngrams):
        words = [] 
        for nWord in range(nGram):
            words = words + [poem[nWord+position]]
        temp_dic = dictionary

        for nWord in range(nGram):
            current_word = words[nWord]
            last_word = nWord+1==nGram
            if(current_word in temp_dic):
                if(last_word):
                    temp_dic[current_word] = temp_dic[current_word]+1 #Increase the Ngram Count by 1
                else:    
                    temp_dic = temp_dic[current_word]
            else:
                create_dic = 0
                if(last_word):
                    create_dic = 1
                else:
                    create_dic = {words[-1]:1}
                
                for k in range(nGram-2,nWord,-1):
                    create_dic = {words[k]:create_dic}
                temp_dic[current_word] = create_dic
                break
    return(dictionary)

In [260]:
dic4 = {}
for i in Poems:
    dic4 = NGram(dic4, i,4)

In [261]:
def Next_word3(word1,word2,word3, dic):
    Max = 0
    next_word = ""
    for i,j in list(dic[word1][word2][word3].items()):
        if(j>Max):
            Max = j
            next_word = i
    return(next_word)
#Next_word3("NEWLINE","love","is",dic4)

In [264]:
prev_word1 = "elmer"
prev_word2 = "is"#"Next_word(prev_word1, dic)"
prev_word3 = "all"# Next_word2(prev_word1,prev_word2, dic2)
generate_str = prev_word1+" "+prev_word2 + " " +prev_word3
for i in range(50):
    next_word = Next_word3(prev_word1, prev_word2,prev_word3, dic4)
    prev_word1 = prev_word2
    prev_word2 = prev_word3
    prev_word3 = next_word
    generate_str = generate_str+" "+ next_word
print(generate_str.replace("NEWLINE ","\n"))

KeyError: 'is'

In [ ]:
# how should i go about generating new lines

In [ ]:
# log the devices being used for tensorflow operations
tf.debugging.set_log_device_placement(True)